In [2]:
# Importing libraries
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [84]:
zhvi_one_bed_metro = pd.read_csv("../data/ZHVI_1_Bed_Metro_US.csv")

In [158]:
# We only want the median income columns from each state
median_income_col = []

# Loop through all columns and only return median income
for info in income_by_state_2022.columns:
    if 'income (dollars)!!Estimate' in info:
        median_income_col.append(info)

In [246]:
# Creating empty data frame to append all the individual csvs
global_income_data = pd.DataFrame()

# Looping through each income data file for years 2010 - 2022
for year in range (2010,2023):
    income_data = pd.read_csv(f'../data/income_by_state/income_by_state_{year}.csv')

    # Subsetting data to only include household income for each state & income by age for each state
    household_income = income_data.loc[:1]
    age_income = income_data.loc[13:16]

    # Combining household & age income
    income_data = pd.concat([household_income,age_income])\
    .loc[1:]\
    .reset_index(drop = True)

    # Make the household & age groups the index
    income_data = income_data.set_index('Label (Grouping)')

    # Subset the data to only include median columns
    income_data = income_data[old_median_income_col]

    # Rearranging the data so it's easier to read
    income_data = income_data.transpose()
    income_data = income_data.reset_index()\
    .rename(columns = {'index':'median_income'})

    # Stripping the column names so they're cleaner
    income_data.columns = [column.strip() for column in income_data.columns]

    # Creating empty list for the states append
    state_list = []

    # Parse the state from the long string
    for label in income_data['median_income']:
        state = label.split('!!')[0]
        state_list.append(state)

    # Making a column for the state
    income_data['state'] = state_list

    # Making a column for the year
    income_data['year'] = year

    # Dropping redundant column
    income_data = income_data.drop(columns = 'median_income')

    # Removing commas from all numeric columns
    income_data['15 to 24 years'] = income_data['15 to 24 years'].str.replace(',','')
    income_data['25 to 44 years'] = income_data['25 to 44 years'].str.replace(',','')
    income_data['45 to 64 years'] = income_data['45 to 64 years'].str.replace(',','')
    income_data['65 years and over'] = income_data['65 years and over'].str.replace(',','')
    income_data['Households'] = income_data['Households'].str.replace(',','')

    #Converting income columns to integers
    income_data['15 to 24 years'] = income_data['15 to 24 years'].astype(int)
    income_data['25 to 44 years'] = income_data['25 to 44 years'].astype(int)
    income_data['45 to 64 years'] = income_data['45 to 64 years'].astype(int)
    income_data['65 years and over'] = income_data['65 years and over'].astype(int)
    income_data['Households'] = income_data['Households'].astype(int)

    # Appending this income data to the global one
    global_income_data = pd.concat([global_income_data, income_data], ignore_index= True)

# Creating csv for median income by state
global_income_data.to_csv("../data/median_income_by_state.csv", index = False)